**Credits:** Help taken from [Abhishek Thakur's notebook](https://www.kaggle.com/abhishek/competition-part-1-baseline/notebook?select=train_folds.csv): 

## Importing Relevant Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

## Reading Data

In [ ]:
df = pd.read_csv("../input/30days-ml-competition-datasets/train_folded.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

## Preprocessing

In [ ]:
useful_features = [col for col in df.columns if col not in ["id", "target", "kfold"]]
df_test = df_test[useful_features]

object_columns = [col for col in df.columns if 'cat' in col]

## Data Modelling and Validation

In [ ]:
final_predictions = []
for fold in range(5):
    x_train =  df[df.kfold != fold].reset_index(drop=True)
    x_valid = df[df.kfold == fold].reset_index(drop=True)
    x_test = df_test.copy()

    y_train = x_train.target
    y_valid = x_valid.target
    
    x_train = x_train[useful_features]
    x_valid = x_valid[useful_features]
    
    ordinal_encoder = OrdinalEncoder()
    x_train[object_columns] = ordinal_encoder.fit_transform(x_train[object_columns])
    x_valid[object_columns] = ordinal_encoder.transform(x_valid[object_columns])
    x_test[object_columns] = ordinal_encoder.transform(x_test[object_columns])
    
    model = XGBRegressor(random_state=fold, n_jobs=4)
    model.fit(x_train, y_train)
    preds_valid = model.predict(x_valid)
    test_preds = model.predict(x_test)
    final_predictions.append(test_preds)
    print(fold, mean_squared_error(y_valid, preds_valid, squared=False))

## Making Submission

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)

In [ ]:
sample_submission.target = preds
sample_submission

In [ ]:
sample_submission.to_csv("submission.csv", index=False)